In [53]:
import pandas as pd

# Import dataset
services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)

# rename variables
# Are still missing the ones related to schedule
newColumns = {'_coordenadas_latitude': 'lat', 
              '_coordenadas_longitude' : 'long',  
              'pais' : 'ubicacion' , 
              'punto_reporte' : 'lugar_enc', 
              'ubicacion' : 'punto_ayud', 
              'cualotropunto' : 'punto_ayud_otro', 
              'infraestructura' : 'cuenta_con' , 
              'sexo' : 'identifica', 
              'organizacionpertence' : 'org_pert', 
              'infraestructuradiscapacidad' : 'serv_disca' , 
              'servicios' : 'serv_tipo',
              'siserviciosnna' : 'serv_dif_n',
              'serviciosnna' : 'cual_serv1',
              'otroservicionna' : 'otro_cual_',
              'siserviciosmujeres' : 'serv_dif_m',
              'serviciosmujeres' : 'cual_ser_2',
              'otroserviciomujeres' : 'otro_dif_m',
              }
services_carto = services.rename(columns=newColumns)
#services_carto.head()

C:\Users\maola\AppData\Local\Temp\ipykernel_12792\246345009.py:4: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)


In [21]:
# serv_tipo id the variable to test (separating by pipe symbol and categorized with number)

services_dict = {
  1 :"asesorialegal",
  2 :"alimentacionynutricion",
  3 :"saludprimerosauxiliosyatencionmedica",
  4 :"aguapotable",
  5 :"saneamiento",
  6 :"higiene",
  7 :"educacion",
  8 :"alojamientotemporal",
  9 :"serviciosdeproteccion",
  10 :"acompañamientoayudapsicosocial",
  11 :"transportehumanitario",
  12 :"restablecimientodecontactofamiliar",
  13:"transferencias",
  14:"Otro",
}

reversed_services_dict = dict( [( x[1],x[0])  for x in services_dict.items()])

def codifyServices(x: str, services_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = services_dict[value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("14")
            
    return "|".join(output)

services_carto["serv_tipo1"] = services_carto["serv_tipo"].apply(lambda x: codifyServices(x, reversed_services_dict))
#services_carto["serv_tipo1"]

In [25]:
# re structure variable cuenta_con
cuenta_con_dict = {
  1 :"accesoaserviciodeinternet",
  2 :"accesoaservicioelectrico",
  3 :"areadecomedorescocina",
  4 :"areaparadisposiciondebasuras",
  5 :"areascomunes",
  6 :"areasdelavanderia",
  7 :"areasderegistrodelapoblacion",
  8 :"ningunadelasanteriores",
}

reversed_cuentacon_dict = dict( [( x[1],x[0])  for x in cuenta_con_dict.items()])

def codifyServices(x: str, cuenta_con_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = cuenta_con_dict [value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("8")
            
    return "|".join(output)

services_carto["cuenta_c_1"] = services_carto["cuenta_con"].apply(lambda x: codifyServices(x, reversed_cuentacon_dict))
#services_carto["cuenta_c_1"]

0    1|3|6
1    1|2|7
2    1|4|7
3      1|2
Name: cuenta_c_1, dtype: object

In [62]:
# re structure variable children services (cual_serv1)
cual_serv1_dict = {
  1 :"nnaalimentacionynutricion",
  2 :"nnaapoyooayudapsicosocial",
  3 :"nnabanosexclusivosnna",
  4 :"nnacuidadosalternativos",
  5 :"nnaentregadekits",
  6 :"espaciosamigables",
  7 :"nnagestiondecasos",
  8 :"nnaotro",
  9 : " ",
}

reversed_cual_serv1_dict = dict( [( x[1],x[0])  for x in cual_serv1_dict.items()])

def codifyServices(x: str, cual_serv1_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = cual_serv1_dict [value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("9")
            
    return "|".join(output)

services_carto["cual_ser_1"] = services_carto["cual_serv1"].astype(str).apply(lambda x: codifyServices(x, reversed_cual_serv1_dict))
services_carto["cual_ser_1"]

0      9
1      1
2      8
3    1|6
Name: cual_ser_1, dtype: object

In [61]:
# re structure variable women services (cual_ser_2)
cual_ser_2_dict = {
  1 :"mujeresapoyonutricionalgestantesylactantes",
  2 :"mujerescontrolesprenatales",
  3 :"mujeresduchasobanos",
  4 :"mujeresespaciosseguros",
  5 :"mujeresgestiondecasosdeviolenciasbasadasengenero",
  6 :"mujeresproductosdehigienemenstrual",
  7 :"mujeresotro",
  8 : " ",
}

reversed_cual_ser_2_dict = dict( [( x[1],x[0])  for x in cual_ser_2_dict.items()])

def codifyServices(x: str, cual_ser_2_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = cual_ser_2_dict [value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("8")
            
    return "|".join(output)

services_carto["cual_ser_3"] = services_carto["cual_ser_2"].astype(str).apply(lambda x: codifyServices(x, reversed_cual_ser_2_dict))
services_carto["cual_ser_3"]


0    3|6
1      1
2    1|2
3    4|5
Name: cual_ser_3, dtype: object

In [58]:
# re structure variable data storage (almacenamientoregistros)
registro_dict = {
  1 :"archivofisico",
  2 :"hojacalculo",
  3 :"software",
  4 :"otroalmacenarregistro",
  5 : " ",
}

reversed_registro_dict  = dict( [( x[1],x[0])  for x in registro_dict.items()])

def codifyServices(x: str, registro_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = registro_dict [value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("5")
            
    return "|".join(output)

services_carto["almacenamientoregistros_"] = services_carto["almacenamientoregistros"].astype(str).apply(lambda x: codifyServices(x, reversed_registro_dict))
services_carto["almacenamientoregistros_"]


0        5
1        2
2    1|2|3
3        2
Name: almacenamientoregistros_, dtype: object